In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
import matplotlib.patches as patches
from pydicom import dcmread, read_file
import skimage as sk
import cv2
import shutil
import os
from tqdm import tqdm

In [10]:
#extraer info de imagenes sin hallazgos para adjuntarlas al dataset
todo= pd.read_csv("/media/deadwolf/BEC4-9B89/chext14/Todas.csv")
bboxes=pd.read_csv("/media/deadwolf/BEC4-9B89/chext14/bboxes.csv")
data=pd.read_csv("/media/deadwolf/BEC4-9B89/chext14/Data.csv")

In [6]:
NoOpacities=data[data['Finding Labels']=='No Finding']

In [8]:
paths=NoOpacities['Image Index']
labels=NoOpacities['Finding Labels']

In [9]:
df = pd.DataFrame({'id': paths, 'label': labels})

In [11]:
df.to_csv('/media/deadwolf/BEC4-9B89/chext14/NoOp.csv', index=False)

In [10]:
info= pd.read_csv("bboxes.csv")
info.head()

,id,label,bboxes
0,00016568_026.png,Pneumothorax,"[[99.6910052910053,91.0222222222222,254.645502..."
1,00025252_032.png,Pneumothorax,"[[756.053333333333,681.851267361111,201.386666..."
2,00001946_029.png,Pneumothorax,"[[289.564444444444,396.269045138889,72.8177777..."
3,00017137_016.png,Pneumothorax,"[[170.097777777778,147.095711805556,287.857777..."
4,00014822_039.png,Pneumothorax,"[[644.551111111111,205.122378472222,194.56,623..."


In [9]:
#Determinemos la cantidad de elementos por etiqueta para distribuirlos equitativamente en los conjuntos de entrenamiento, prueba y validación
# train 0.6  validation 0.3 test 0.1
conteo = info['label'].value_counts()
conteo

label
Atelectasis     180
Effusion        153
Cardiomegaly    146
Infiltrate      123
No Finding      122
Pneumonia       120
Pneumothorax     98
Mass             85
Nodule           79
Name: count, dtype: int64

In [22]:
# reformateando imagenes del tipo (1024,1024,n) a (1024,1024)
root='/home/jair/COVID/NIHX14/images1024/'
for idd in info['id']:
    r=root+idd
    img=sk.io.imread(r)
    if img.shape == (1024,1024):
        pass
    else:
        imgr=img[...,0]
        sk.io.imsave('/home/jair/COVID/NIHX14/'+idd, imgr)

In [32]:
#mover a carpetas de train / test / validation (recordemos que una imagen puede presentar dos clases distintas)
rootd='/home/jair/COVID/NIHX14/images/1024/'
rooto='/home/jair/COVID/NIHX14/images1024/'
mode='test'
csv=pd.read_csv(mode+"_ord.csv")
for idd in tqdm(csv['id']):
    if os.path.exists(rootd+mode+'/'+idd):
        pass
    else:
        rd=rootd+mode
        ro=rooto+idd
        shutil.copy(ro, rd)

100%|███████████████████████████████████████| 109/109 [00:00<00:00, 1438.15it/s]


In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
import matplotlib.patches as patches
from pydicom import dcmread, read_file
import skimage as sk
import cv2
import shutil
import os
from tqdm import tqdm

mode='test'
ro='/home/jair/COVID/NIHX14/images/1024/'+mode+'/'
rd='/home/jair/COVID/NIHX14/datasets/yolo/640/images/'+mode+'/'
ra='/home/jair/COVID/NIHX14/'+mode+'_ord.csv'
anno={'Atelectasis':0,'Effusion':1,'Cardiomegaly':2,'Infiltrate':3,'No Finding':4,'Pneumonia':5,'Pneumothorax':6,'Mass':7,'Nodule':8}

info=pd.read_csv(ra)

#funcion para pasar bbox a formato de yolo
def xywh_yolo(bb,size):
    cx=bb[0]+bb[2]/2
    cy=bb[1]+bb[3]/2
    xc=cx/size
    yc=cy/size
    wc=bb[2]/size
    hc=bb[3]/size
    return [xc,yc,wc,hc]

#redimensionando imagenes y creando dataset para YOLO
for indice, fila in tqdm(info.iterrows()):
    ri=ro+fila['id']
    img=sk.io.imread(ri)
    imgr=sk.transform.resize(img,(640,640))
    rgb = np.zeros((640,640,3))
    rgb[...,0] =imgr
    rgb[...,1] =imgr
    rgb[...,2] =imgr
    rgb=(rgb*255).astype(np.uint8)
    sk.io.imsave(rd+fila['id'],rgb)
    for bbox in literal_eval(fila['bboxes']):
        #que los transforme del tamaño de imagenes originales normalizados ya no importa escala solo al recuperarlos multiplicar por escala adecuada
        bboxy=xywh_yolo(bbox,1024)
        with open(rd.replace('images','labels')+fila['id'].replace('png','txt'), 'a') as file:
            if fila['label'] !='No Finding':
                file.write(str(anno[fila['label']])+' '+str(bboxy[0])+' '+str(bboxy[1])+' '+str(bboxy[2])+' '+str(bboxy[3])+'\n')
            else:
                file.write(str(anno[fila['label']])+' 0.500781 0.500781 0.9946 0.9946\n')

109it [00:16,  6.55it/s]


In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
import matplotlib.patches as patches
from pydicom import dcmread, read_file
import skimage as sk
import cv2
import shutil
import os
from tqdm import tqdm

mode='train'
ro='/home/jair/COVID/NIHX14/images/1024/'+mode+'/'
rd='/home/jair/COVID/NIHX14/datasets/yolo/640/images/'+mode+'/'
ra='/home/jair/COVID/NIHX14/'+mode+'_ord.csv'
anno={'Atelectasis':0,'Effusion':1,'Cardiomegaly':2,'Infiltrate':3,'No Finding':4,'Pneumonia':5,'Pneumothorax':6,'Mass':7,'Nodule':8}

info=pd.read_csv(ra)

def xywh_yolo(bb,size):
    cx=bb[0]+bb[2]/2
    cy=bb[1]+bb[3]/2
    xc=cx/size
    yc=cy/size
    wc=bb[2]/size
    hc=bb[3]/size
    return [xc,yc,wc,hc]

def yolon_to_xywh(bb,size):
    x=bb[0]*size-bb[2]*size/2
    y=bb[1]*size-bb[3]*size/2
    wp=bb[2]*size
    hp=bb[3]*size
    return [x, y, wp, hp]

def xywh_xiyixfyf(bbox):
    x1=bbox[0]
    y1=bbox[1]
    x2=bbox[0]+bbox[2]
    y2=bbox[1]+bbox[3]
    return [x1,y1,x2,y2]

#crear csv para retinanet
data = pd.DataFrame(columns=['path','x1','y1','x2','y2','class'])
pth=[]
x1=[]
y1=[]
x2=[]
y2=[]
c=[]
for indice, fila in tqdm(info.iterrows()):
    for bbox in literal_eval(fila['bboxes']):
        pth.append(rd+fila['id'])
        bboxy=xywh_yolo(bbox,1024)
        bboxo=yolon_to_xywh(bbox,640)
        bboxr=xywh_xiyixfyf(bboxy)
        if fila['label'] !='No Finding':
            x1.append(int(bboxr[0]))
            y1.append(int(bboxr[1]))
            x2.append(int(bboxr[2]))
            y2.append(int(bboxr[3]))
            c.append(fila['label'])
        else:
            x1.append(0)
            y1.append(0)
            x2.append(640)
            y2.append(640)
            c.append(fila['label'])
         


data['path']=pth
data['x1']=x1
data['y1']=y1
data['x2']=x2
data['y2']=y2
data['class']=c

data=data.sample(frac=1).reset_index(drop=True)
data.to_csv('/home/jair/COVID/NIHX14/datasets/retinanet/640/'+mode+'.csv', index=False)

0it [00:00, ?it/s]

[149.152542372881, 518.815826933263, 112.813559322034, 45.5593220338983]
[59357.28813559296, 317463.14618644083, 72200.67796610176, 29157.96610169491]
[59357.28813559296, 317463.14618644083, 131557.96610169474, 346621.11228813574]
